In [7]:
# import torch
# import math
# from torch import nn
# import copy
# import torch.nn.functional as F
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#
# def attention(query, key, value, mask=None, dropout=None):
#     """
#     Scaled dot product attention
#     query: (B, h, T_q, d_k)
#     key: (B, h, T_k, d_k)
#     value: (B, h, T_k, d_k)
#     mask: broadcastable to (B, h, T_q, T_k) with True where allowed (or 1)
#     returns: (B, h, T_q, d_k), attn (B, h, T_q, T_k)
#     """
#     d_k = query.size(-1)
#     scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
#     if mask is not None:
#         scores = scores.masked_fill(mask==0,-1e9)
#     p_attn = nn.functional.softmax(scores, dim=-1)
#     if dropout is not None:
#         p_attn = dropout(p_attn)
#     return torch.matmul(p_attn, value), p_attn
#
# from copy import deepcopy
# class MultiHeadedAttention(nn.Module):
#     def __init__(self, h, d_model, dropout=0.1):
#         """
#         h: number of heads
#         d_model: model dimensionality (must be divisible by h)
#         """
#         super().__init__()
#         assert d_model % h == 0
#         self.d_k = d_model // h
#         self.h = h
#         self.linears = nn.ModuleList([deepcopy(
#             nn.Linear(d_model, d_model)) for i in range(4
#         )])
#         self.attn = None
#         self.dropout = nn.Dropout(p=dropout)
#
#     def forward(self, query, key, value, mask=None):
#         if mask is not None:
#             mask = mask.unsqueeze(1)
#         nbatches = query.size(0)
#         query, key, value = [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1,2) for
#             l,x in zip(self.linears, (query, key, value))]
#         x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
#         x = x.transpose(1,2).contiguous().view(nbatches, -1, self.h*self.d_k)
#         output = self.dropout(x)
#         return output
#
# class PositionwiseFeedForward(nn.Module):
#     def __init__(self, d_model, d_ff, dropout=0.1):
#         super().__init__()
#         self.w_1 = nn.Linear(d_model, d_ff)
#         self.w_2 = nn.Linear(d_ff, d_model)
#         self.dropout = nn.Dropout(dropout)
#     def forward(self, x):
#         h1 = self.w_1(x)
#         h2 = self.dropout(h1)
#         return self.w_2(h2)
#
# class EncoderLayer(nn.Module):
#     def __init__(self, size, self_attn, feed_forward, dropout):
#         super().__init__()
#         self.self_attn = self_attn
#         self.feed_forward = feed_forward
#         self.sublayer = nn.ModuleList([deepcopy(
#             SublayerConnection(size, dropout)) for i in range(2)
#         ])
#         self.size = size
#     def forward(self, x, mask):
#         x = self.sublayer[0](
#             x, lambda x: self.self_attn(x, x, x, mask)
#         )
#         output = self.sublayer[1](x, self.feed_forward)
#         return output
#
# class SublayerConnection(nn.Module):
#     def __init__(self, size, dropout):
#         super().__init__()
#         self.norm = LayerNorm(size)
#         self.dropout = nn.Dropout(dropout)
#     def forward(self, x, sublayer):
#         output = x + self.dropout(sublayer(self.norm(x)))
#         return output
#
# class LayerNorm(nn.Module):
#     def __init__(self, features, eps=1e-6):
#         super().__init__()
#         self.a_2 = nn.Parameter(torch.ones(features))
#         self.b_2 = nn.Parameter(torch.zeros(features))
#         self.eps = eps
#     def forward(self, x):
#         mean = x.mean(-1, keepdim=True)
#         std = x.std(-1, keepdim=True)
#         x_zscore = (x-mean) / torch.sqrt(std**2 + self.eps)
#         output = self.a_2 * x_zscore + self.b_2
#         return output
#
# from copy import deepcopy
# class Encoder(nn.Module):
#     def __init__(self, layer, N):
#         super().__init__()
#         self.layers = nn.ModuleList([deepcopy(layer) for i in range(N)])
#         self.norm = LayerNorm(layer.size)
#     def forward(self, x, mask):
#         for layer in self.layers:
#             x = layer(x, mask)
#             output = self.norm(x)
#         return output
#
# class DecoderLayer(nn.Module):
#     def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
#         super().__init__()
#         self.size = size
#         self.self_attn = self_attn
#         self.src_attn = src_attn
#         self.feed_forward = feed_forward
#         self.sublayer = nn.ModuleList([deepcopy(SublayerConnection(size, dropout)) for
#                                        i in range(3)])
#     def forward(self, x, memory, src_mask, tgt_mask):
#         x = self.sublayer[0](x, lambda x:
#                              self.self_attn(x, x, x, tgt_mask))
#         x = self.sublayer[1](x, lambda x:
#                              self.src_attn(x, memory, memory, src_mask))
#         output = self.sublayer[2](x, self.feed_forward)
#         return output
#
# class Decoder(nn.Module):
#     def __init__(self, layer, N):
#         super().__init__()
#         self.layers = nn.ModuleList([deepcopy(layer) for i in range(N)])
#         self.norm = LayerNorm(layer.size)
#     def forward(self, x, memory, src_mask, tgt_mask):
#         for layer in self.layers:
#             x = layer(x, memory, memory, src_mask, tgt_mask)
#         output = self.norm(x)
#         return output
#
#
# class Transformer(nn.Module):
#     def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
#         super().__init__()
#         self.encoder = encoder
#         self.decoder = decoder
#         self.src_embed = src_embed
#         self.tgt_embed = tgt_embed
#         self.generator = generator
#     def encode(self, src, src_mask):
#         return self.encoder(self.src_embed(src), src_mask)
#     def decode(self, memory, src_mask, tgt, tgt_mask):
#         return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
#     def forward(self, src, tgt, src_mask, tgt_mask):
#         memory = self.encode(src, src_mask)
#         output = self.decode(memory, src_mask, tgt, tgt_mask)
#         return output
#
# class Generator(nn.Module):
#     def __init__(self, d_model, vocab):
#         super().__init__()
#         self.proj = nn.Linear(d_model, vocab)
#     def forward(self, x):
#         out = self.proj(x)
#         probs = nn.functional.log_softmax(out, dim=-1)
#         return probs
#
# def create_model(src_vocab, tgt_vocab, N, d_model, d_ff, h, dropout=0.1):
#     attn = MultiHeadedAttention(h, d_model).to(DEVICE)
#     ff = PositionwiseFeedForward(d_model, d_ff, dropout).to(DEVICE)
#     pos = PositionalEncoding(d_model, dropout).to(DEVICE)
#     model = Transformer(
#         Encoder(EncoderLayer(d_model, deepcopy(attn), deepcopy(ff),
#                              dropout).to(DEVICE), N).to(DEVICE)
#         Decoder(DecoderLayer(d_model, deepcopy(attn),
#                              deepcopy(attn), deepcopy(ff), dropout).to(DEVICE),N
#     ).to(DEVICE),
#         nn.Sequential(Embeddings(d_model, src_vocab).to(DEVICE), deepcopy(pos)),
#         nn.Sequential(Embeddings(d_model, tgt_vocab).to(DEVICE), deepcopy(pos)),
#         Generator(d_model, tgt_vocab).to(DEVICE)
#     for p in model.parameters():
#         if p.dim() > 1:
#             nn.init.xvavier_uniform_(p)
#     return model.to(DEVICE)



In [8]:
# transformer_model.py
import math, copy
import torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def attention(query, key, value, mask=None, dropout=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # ensure boolean mask where True means "allowed"
        if mask.dtype != torch.bool:
            mask = mask != 0
        # mask shape should broadcast to (B, h, T_q, T_k); use ~mask because masked_fill expects True where to fill
        print("mask matrix")
        print(mask)
        scores = scores.masked_fill(~mask, -1e9)
        print(scores)
    p_attn = F.softmax(scores, dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0
        self.h = h
        self.d_k = d_model // h
        self.linears = nn.ModuleList([copy.deepcopy(nn.Linear(d_model, d_model)) for _ in range(4)])
        self.attn = None
        self.dropout = nn.Dropout(dropout)
    def forward(self, query, key, value, mask=None):
        breakpoint()
        if mask is not None:
            # if mask is (B, T_q, T_k) -> make it (B, 1, T_q, T_k) for broadcasting
            if mask.dim() == 3:
                # unsqueeze returns a tensor with a dimension one inserted at specified position
                mask = mask.unsqueeze(1)
        B = query.size(0)
        # Project and split heads
        # T is time steps or sequence length, "Hello World" T = 2
        # h is number of attention heads, common values 8, 12, 16, BERT is 12
        # T_q is length of query sequence
        # T_k is length of key sequence
        # d_k is dimension per head, d_model / h
        # d_model is the model dimension
        query, key, value = [
            # linear projection of x
            # B, T, h, d_k
            # B, h, T, d_k
            lin(x).view(B, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears[:3], (query, key, value))
        ]
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        # after attention is B, h, T, d_k
        # transpose B, T, h, d_k
        # d_model is h * d_k , and output B, T, d_model
        x = x.transpose(1, 2).contiguous().view(B, -1, self.h * self.d_k)
        return self.linears[3](x)

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, keepdim=True, unbiased=False)
        x_norm = (x - mean) / torch.sqrt(var + self.eps)
        return self.a_2 * x_norm + self.b_2

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(2)])
        self.size = size
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class Encoder(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)])
        self.norm = LayerNorm(layer.size)
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super().__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = nn.ModuleList([copy.deepcopy(SublayerConnection(size, dropout)) for _ in range(3)])
    def forward(self, x, memory, src_mask, tgt_mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, memory, memory, src_mask))
        return self.sublayer[2](x, self.feed_forward)

class Decoder(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = nn.ModuleList([copy.deepcopy(layer) for _ in range(N)])
        self.norm = LayerNorm(layer.size)
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)
    def forward(self, src, tgt, src_mask, tgt_mask):
        memory = self.encode(src, src_mask)
        return self.decode(memory, src_mask, tgt, tgt_mask)

class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab)
    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super().__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :].to(x.device)
        return self.dropout(x)

def make_model(src_vocab, tgt_vocab, N=2, d_model=32, d_ff=64, h=4, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model, dropout)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    pos = PositionalEncoding(d_model, dropout)
    model = Transformer(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(pos)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(pos)),
        Generator(d_model, tgt_vocab)
    ).to(DEVICE)

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model


In [9]:
# make_dummy_data.py
import torch
import random
import numpy as np

SEED = 42
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

B = 2           # batch size
T_src = 5       # source sequence length
T_tgt = 6       # target sequence length
SRC_VOCAB = 11
TGT_VOCAB = 11

# tokens 0..vocab-1 (we reserve 0 for padding in general)
src = torch.randint(1, SRC_VOCAB, (B, T_src), dtype=torch.long)
tgt = torch.randint(1, TGT_VOCAB, (B, T_tgt), dtype=torch.long)

# masks: 1 where real token, 0 where padding (here no padding)
src_mask = (src != 0).unsqueeze(-2)       # (B, 1, T_src)
tgt_mask = (tgt != 0).unsqueeze(-2)      # (B, 1, T_tgt)

# subsequent_mask for causal decoding
def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent = torch.triu(torch.ones(attn_shape), diagonal=1).bool()
    return ~subsequent

sub = subsequent_mask(T_tgt)              # (1, T_tgt, T_tgt)
tgt_mask = tgt_mask.unsqueeze(1) & sub   # (B, 1, T_tgt, T_tgt)

torch.save({
    'src': src,
    'tgt': tgt,
    'src_mask': src_mask,
    'tgt_mask': tgt_mask,
    'src_vocab': SRC_VOCAB,
    'tgt_vocab': TGT_VOCAB
}, 'data.pt')

print("Saved data.pt with shapes:")
print("src", src.shape, "tgt", tgt.shape, "src_mask", src_mask.shape, "tgt_mask", tgt_mask.shape)


Saved data.pt with shapes:
src torch.Size([2, 5]) tgt torch.Size([2, 6]) src_mask torch.Size([2, 1, 5]) tgt_mask torch.Size([2, 1, 6, 6])


In [10]:
# run_smoketest.py
import torch
# from transformer_model import make_model, PositionalEncoding, Embeddings, Generator, DEVICE

data = torch.load('data.pt')
src = data['src'].to(DEVICE)
tgt = data['tgt'].to(DEVICE)
src_mask = data['src_mask'].to(DEVICE)
tgt_mask = data['tgt_mask'].to(DEVICE)
src_vocab = data['src_vocab']
tgt_vocab = data['tgt_vocab']

print("Device:", DEVICE)
print("Loaded src shape:", src.shape, "tgt shape:", tgt.shape)
print("src_mask", src_mask.shape, "tgt_mask", tgt_mask.shape)

# Build the small model (matching sizes from transformer_model.py make_model defaults)
model = make_model(src_vocab, tgt_vocab, N=2, d_model=32, d_ff=64, h=4, dropout=0.1)
model.train()

out = model(src, tgt, src_mask, tgt_mask)  # (B, T_tgt, d_model)
print("Output shape:", out.shape)

# generator to log-probabilities
log_probs = model.generator(out)  # (B, T_tgt, V)
print("Log probs shape:", log_probs.shape)

# quick numerical checks
print("NaN in outputs:", torch.isnan(log_probs).any().item())
print("Inf in outputs:", torch.isinf(log_probs).any().item())

# toy target and loss
targets = torch.randint(0, tgt_vocab, (src.size(0), tgt.size(1))).to(DEVICE)
loss_fn = torch.nn.NLLLoss()
loss = loss_fn(log_probs.view(-1, tgt_vocab), targets.view(-1))
print("Loss:", loss.item())
loss.backward()
print("Backward OK. Gradient present on first param:", next(model.parameters()).grad is not None)




Device: cpu
Loaded src shape: torch.Size([2, 5]) tgt shape: torch.Size([2, 6])
src_mask torch.Size([2, 1, 5]) tgt_mask torch.Size([2, 1, 6, 6])
mask matrix
tensor([[[[True, True, True, True, True]]],


        [[[True, True, True, True, True]]]])
tensor([[[[-0.4995,  0.1880, -3.3225,  0.0419, -2.6722],
          [-0.1723,  0.4228, -0.7855, -0.8842, -1.0246],
          [-0.3008,  0.5816, -1.0868,  0.4911, -0.6556],
          [ 1.0010, -0.2941, -0.3300, -0.8776, -0.1582],
          [-0.3688,  0.8469, -2.0946,  0.2840, -1.4589]],

         [[ 0.0201,  0.6344, -0.8660,  0.9146, -1.4362],
          [ 1.9139, -0.6253, -0.3232, -1.5265,  1.0153],
          [ 1.0377,  0.2003, -0.0792,  0.2176,  0.1688],
          [ 1.1756,  0.8227, -0.7845,  0.7713, -0.2874],
          [ 1.0643,  1.4644, -1.2174, -0.0057, -0.5145]],

         [[ 0.0073, -0.1851,  1.0104, -0.2575,  1.1681],
          [-1.0207,  1.3500, -1.1273, -0.1228, -1.5513],
          [ 0.7092, -0.4316,  0.6800, -0.6082,  0.7532],
        